# GridSearch of Multi-Layer Perceptron Classifier Model for Injection Molding of Bins using tsfresh data

In [1]:
import pandas as pd
import numpy as np
import my_preprocessing_functions as mpf
from sklearn.metrics import confusion_matrix, recall_score, f1_score, precision_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sn
from pprint import pp
import os

In [2]:
filename = 'processed_data_pv_0.06.csv'
data_path = os.getcwd() + "/BalanceData/"
dataset_df = pd.read_csv(data_path + filename)

In [3]:
print('For the full dataset,')
mpf.check_imbalance(dataset_df['valid_or_not'])

For the full dataset,
The imbalance profile has:
 50.00% of valid products and     
 50.00% of invalid products


In [4]:
X = dataset_df.drop('valid_or_not', axis='columns')
y = dataset_df['valid_or_not']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [6]:
mlp_gs = MLPClassifier(activation='relu', hidden_layer_sizes = (537, 537, 537), max_iter=500, random_state=123)
parameter_space = {
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
}
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=2, verbose=3)
clf.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 2/2] END alpha=0.0001, learning_rate=constant, solver=sgd;, score=0.642 total time=  48.6s
[CV 2/2] END alpha=0.0001, learning_rate=adaptive, solver=adam;, score=0.696 total time= 1.3min
[CV 2/2] END alpha=0.0001, learning_rate=constant, solver=adam;, score=0.696 total time= 1.3min
[CV 1/2] END alpha=0.0001, learning_rate=adaptive, solver=adam;, score=0.747 total time= 1.4min
[CV 1/2] END alpha=0.0001, learning_rate=constant, solver=adam;, score=0.747 total time= 1.5min
[CV 1/2] END alpha=0.0001, learning_rate=constant, solver=sgd;, score=0.764 total time= 1.7min
[CV 1/2] END alpha=0.05, learning_rate=constant, solver=adam;, score=0.669 total time=  47.4s
[CV 2/2] END alpha=0.05, learning_rate=constant, solver=sgd;, score=0.603 total time=  51.6s
[CV 1/2] END alpha=0.05, learning_rate=constant, solver=sgd;, score=0.631 total time= 1.6min
[CV 1/2] END alpha=0.0001, learning_rate=adaptive, solver=sgd;, score=0.670 total time

GridSearchCV(cv=2,
             estimator=MLPClassifier(hidden_layer_sizes=(537, 537, 537),
                                     max_iter=500, random_state=123),
             n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.05],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['sgd', 'adam']},
             verbose=3)

In [7]:
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'alpha': 0.0001, 'learning_rate': 'constant', 'solver': 'adam'}


In [8]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

0.703 (+/-0.122) for {'alpha': 0.0001, 'learning_rate': 'constant', 'solver': 'sgd'}
0.721 (+/-0.051) for {'alpha': 0.0001, 'learning_rate': 'constant', 'solver': 'adam'}
0.680 (+/-0.020) for {'alpha': 0.0001, 'learning_rate': 'adaptive', 'solver': 'sgd'}
0.721 (+/-0.051) for {'alpha': 0.0001, 'learning_rate': 'adaptive', 'solver': 'adam'}
0.617 (+/-0.028) for {'alpha': 0.05, 'learning_rate': 'constant', 'solver': 'sgd'}
0.698 (+/-0.059) for {'alpha': 0.05, 'learning_rate': 'constant', 'solver': 'adam'}
0.678 (+/-0.003) for {'alpha': 0.05, 'learning_rate': 'adaptive', 'solver': 'sgd'}
0.698 (+/-0.059) for {'alpha': 0.05, 'learning_rate': 'adaptive', 'solver': 'adam'}


In [9]:
y_true, y_pred = y_test, clf.predict(X_test)
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.78      0.61      0.69       199
           1       0.72      0.85      0.78       233

    accuracy                           0.74       432
   macro avg       0.75      0.73      0.73       432
weighted avg       0.75      0.74      0.74       432

